In [117]:
# Get the data source and prepare a dataframe

import pandas as pd
import numpy as np

data = pd.read_csv("./Dataset/MarchmadnessDatasetFull.csv", header="infer")
# display(data.head(10))

data2 = pd.read_csv("./Dataset/RegularSeasonDetailedResults.csv", header="infer")
display(data2.head(10))

teams = pd.read_csv("./Dataset/Teams.csv", header="infer")
# teams.head(10)




,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [118]:
# Preprocess Dataset

# Drop the Location, Rk and Region info because it's irrelevant
#data2 = data2.drop('DayNum', axis=1)#not important for analysis
data2 = data2.drop('WLoc', axis=1)#not important for analysis
# data2 = data2.drop('WTeamID', axis=1)#not important for analysis
# data2 = data2.drop('LTeamID', axis=1)#not important for analysis
# data2 = data2.drop('Season', axis=1)#not important for analysis

data = data.drop('Rk', axis=1)#not important for analysis
data = data.drop('Year', axis=1)#not important for analysis
data = data.drop('Date', axis=1)#not important for analysis
data = data.drop('Location', axis=1)#not important for analysis
data = data.drop('Region', axis=1)#not important for analysis

data2 = data2.drop('WPF', axis=1) #Personal Fouls are not significant
data2 = data2.drop('LPF', axis=1) 

In [119]:
# data2 = data2.drop('WOR', axis=1)
# data2 = data2.drop('WDR', axis=1)
# data2 = data2.drop('LOR', axis=1)
# data2 = data2.drop('LDR', axis=1)

#
# Further pre-process by appending columns 
#
WFTP = [] #Winning team Free Throw Percentage
WFGP = [] #Winning team Field Goal Percentage
LFTP = [] #Losing team Free Throw Percentage
LFGP = [] #Losing team Field Goal Percentage

# Possession stat formula --  0.96*[(FGA)+(TO)+0.44*(FTA)-(ORB)]
LPoss = []
WPoss = []

# calculate the FG% and FT% and append to dataframe
for index, row in data2.iterrows():
    WFGMade = row['WFGM']
    LFGMade = row['LFGM']
    
    WTurnOver = row['WTO']
    LTurnOver = row['LTO']
    
    WOffRebound = row['WOR']
    LOffRebound = row['LOR']
    
    WFGAttempt = row['WFGA']
    LFGAttempt = row['LFGA']
    
    # calculate percentages
    WFGPercent = (WFGMade/(WFGAttempt+WFGMade))
    LFGPercent = (LFGMade/(LFGAttempt+LFGMade))
    
    
    WFGAttempt = float(WFGAttempt)
    WTurnOver = float(WTurnOver)
    WOffRebound = float(WOffRebound)

    LFGAttempt = float(LFGAttempt)
    LTurnOver = float(LTurnOver)
    LOffRebound = float(LOffRebound)
    
    ##Very important statistic
    WinPoss = (0.96*((WFGAttempt)+(WTurnOver)+0.44*(WFTAttempt)-(WOffRebound)))
    LosPoss = (0.96*((LFGAttempt)+(LTurnOver)+0.44*(LFTAttempt)-(LOffRebound)))
    
    
    # append to list
    WFGP.append(WFGPercent)
    LFGP.append(LFGPercent)
    
    LPoss.append(LosPoss)
    WPoss.append(WinPoss)
    
for index, row in data2.iterrows():
    WFTMade = row['WFTM']
    LFTMade = row['LFTM']
    WFTAttempt = row['WFTA']
    LFTAttempt = row['LFTA']
    WFTPercent = (WFTMade/(WFTAttempt+WFTMade))
    LFTPercent = (LFTMade/(LFTAttempt+LFTMade))
    WFTP.append(WFTPercent)
    LFTP.append(LFTPercent)
    
# Make each list a pandas series 
LfreethrowPercent = pd.Series(LFTP)
WfreethrowPercent = pd.Series(WFTP)

LfieldGoalPercent = pd.Series(LFGP)
WfieldGoalPercent = pd.Series(WFGP)

WinPossession = pd.Series(LPoss)
LosPossession = pd.Series(WPoss)

# append to dataframe as column
data2['WFTP'] = WfreethrowPercent
data2['LFTP'] = LfreethrowPercent
data2['WFGP'] = WfieldGoalPercent
data2['LFGP'] = LfieldGoalPercent

data2['LPoss'] = LosPossession
data2['WPoss'] = WinPossession

data2.head(10)

H:\Tools\Anaconda\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in longlong_scalars
H:\Tools\Anaconda\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in longlong_scalars


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LAst,LTO,LStl,LBlk,WFTP,LFTP,WFGP,LFGP,LPoss,WPoss
0,2003,10,1104,68,1328,62,0,27,58,3,...,8,18,9,2,0.379310,0.421053,0.317647,0.293333,72.3456,61.9392
1,2003,10,1272,70,1393,63,0,26,62,8,...,7,12,8,6,0.344828,0.310345,0.295455,0.263736,65.6256,60.0192
2,2003,11,1266,73,1437,61,0,24,58,8,...,9,12,2,5,0.369565,0.378378,0.292683,0.231579,56.9856,55.2192
3,2003,11,1296,56,1457,50,0,18,38,3,...,9,19,4,3,0.354167,0.347826,0.321429,0.268657,50.2656,52.3392
4,2003,11,1400,77,1208,71,0,30,61,6,...,12,10,7,1,0.458333,0.386364,0.329670,0.279070,63.7056,52.3392
5,2003,11,1458,81,1186,55,0,26,57,6,...,8,19,4,3,0.460000,0.413793,0.313253,0.303030,59.8656,60.0192
6,2003,12,1161,80,1236,62,0,23,55,2,...,11,30,10,4,0.450704,0.416667,0.294872,0.316667,64.6656,62.8992
7,2003,12,1186,75,1457,61,0,28,62,4,...,10,15,14,8,0.416667,0.425000,0.311111,0.253165,73.3056,66.7392
8,2003,12,1194,71,1156,66,0,28,58,5,...,13,25,8,2,0.357143,0.307692,0.325581,0.315789,71.3856,64.8192
9,2003,12,1458,84,1296,56,0,32,67,5,...,10,18,1,3,0.441176,0.368421,0.323232,0.306667,64.6656,61.9392


In [120]:


print(WinPoss)

56.9856


In [121]:
#Remove teams that weren't in LastD1Season as they aren't in the tourney

real_teams = ['Alabama', 'Arizona','Arizona St', 'Arkansas', 'Auburn','Bucknell','Buffalo','Butler','CS Fullerton',
'Charleston','Cincinnati', 'Clemson','Creighton', 'Davidson','Duke','Florida','Florida St','Georgia St', 'Gonzaga',
'Houston','Iona','Kansas', 'Kansas State', 'Kentucky','Lipscomb', 'Long Island', 'Loyola-Chicago', 'Marshall',
'Miami FL','Michigan','Michigan St', 'Missouri','Montana','Murray St', 
'Nevada','New Mexico St','NC Central', 'NC State','North Carolina','Ohio St','Oklahoma',
'Penn','Providence', 'Purdue','Radford', 'Rhode Island', 'San Diego St', 'Seton Hall','S Dakota St','St Bonaventure',
'SF Austin','Syracuse','TCU','Tennessee','Texas','Texas A&M', 'TX Southern','Texas Tech','UCLA','UMBC', 
'UNC Greensboro', 'Villanova','Virginia','Virginia Tech', 'West Virginia','Wichita St' ,'Wright St','Xavier'] 


"""#df = df[~df['date'].isin(a)]
teams = teams[teams['TeamName'].isin(real_teams)]
# print((teams))
print(type(teams))
#Now we have current season teams
print(len(data2))
team_dict = dict()
for index, team in teams.iterrows():
    #print(team.TeamID)
    team_dict_list = []
    team_ft_list = []
    team_fgp_list = []
    for index, row2 in data2.iterrows():
        if(row2.WTeamID == team.TeamID):
            #print(row2.WScore)
            team_dict_list.append(row2.WScore - row2.LScore)
            
            team_ft_list.append('%.3f' % round(row2.WFTP,3))
            team_fgp_list.append('%.3f' % round(row2.WFGP,3))
        elif(row2.LTeamID == team.TeamID):
            team_dict_list.append(row2.LScore - row2.WScore)
            team_ft_list.append('%.3f' % round(row2.LFTP,3))
            team_fgp_list.append('%.3f' % round(row2.LFGP,3))
            #print(row2.WScore)
    team_dict[team.TeamName] = {}
    team_dict[team.TeamName]['team_points'] = team_dict_list
    team_dict[team.TeamName]['team_ft_list'] = team_ft_list
    team_dict[team.TeamName]['team_fgp_list'] = team_fgp_list
"""    


"#df = df[~df['date'].isin(a)]\nteams = teams[teams['TeamName'].isin(real_teams)]\n# print((teams))\nprint(type(teams))\n#Now we have current season teams\nprint(len(data2))\nteam_dict = dict()\nfor index, team in teams.iterrows():\n    #print(team.TeamID)\n    team_dict_list = []\n    team_ft_list = []\n    team_fgp_list = []\n    for index, row2 in data2.iterrows():\n        if(row2.WTeamID == team.TeamID):\n            #print(row2.WScore)\n            team_dict_list.append(row2.WScore - row2.LScore)\n            \n            team_ft_list.append('%.3f' % round(row2.WFTP,3))\n            team_fgp_list.append('%.3f' % round(row2.WFGP,3))\n        elif(row2.LTeamID == team.TeamID):\n            team_dict_list.append(row2.LScore - row2.WScore)\n            team_ft_list.append('%.3f' % round(row2.LFTP,3))\n            team_fgp_list.append('%.3f' % round(row2.LFGP,3))\n            #print(row2.WScore)\n    team_dict[team.TeamName] = {}\n    team_dict[team.TeamName]['team_points'] = team_di

In [145]:
#display(data2.head(10))
data2.shape

winThrow = (data2['WFTP'])
loseThrow = (data2['LFTP'])

print(data2.columns.get_loc("WFGP"))
print(data2.columns.get_loc("LFGP"))



diff = pd.DataFrame((data2.iloc[1:,31].values-data2.iloc[:76635,32].values), 
                            columns=['diff_FTP'], index=data2.iloc[1:,0])

diff['diff_FGP'] = pd.DataFrame((data2.iloc[1:,33].values-data2.iloc[:76635,34].values))


display(diff.head(10))







33
34


,diff_FTP,diff_FGP
Season,,
2003,-0.076225,-0.010694
2003,0.059220,-0.010694
2003,-0.024212,-0.010694
2003,0.110507,-0.010694
2003,0.073636,-0.010694
2003,0.036911,-0.010694
2003,0.000000,-0.010694
2003,-0.067857,-0.010694
2003,0.133484,-0.010694


In [179]:
import math
Z = diff['diff_FTP']
count = 0
display(Z.head(10))
for a in Z:
    count += 1
    if(a != a):
        a = float(1)
        #print("NAN UH OH")
        #print("count", count)
for a in Z:
    count += 1
    if(a != a):
        #a = float(1)
        #print("NAN UH OH")
        #print("count", count)    




Season
2003   -0.076225
2003    0.059220
2003   -0.024212
2003    0.110507
2003    0.073636
2003    0.036911
2003    0.000000
2003   -0.067857
2003    0.133484
2003    0.040670
Name: diff_FTP, dtype: float64

NAN UH OH
count 76762
NAN UH OH
count 81846
NAN UH OH
count 85287
NAN UH OH
count 86196
NAN UH OH
count 89282
NAN UH OH
count 90690
NAN UH OH
count 92272
NAN UH OH
count 94510
NAN UH OH
count 94934
NAN UH OH
count 101577
NAN UH OH
count 101670
NAN UH OH
count 103876
NAN UH OH
count 104070
NAN UH OH
count 104620
NAN UH OH
count 104715
NAN UH OH
count 106629
NAN UH OH
count 109672
NAN UH OH
count 110868
NAN UH OH
count 116631
NAN UH OH
count 122113
NAN UH OH
count 126374
NAN UH OH
count 127796
NAN UH OH
count 129764
NAN UH OH
count 132062
NAN UH OH
count 132153
NAN UH OH
count 135135
NAN UH OH
count 141288
NAN UH OH
count 142321
NAN UH OH
count 146590
NAN UH OH
count 149063
NAN UH OH
count 152958
NAN UH OH
count 153003


In [217]:

Z = Z.fillna(0)

from numpy.linalg import inv
    
Z2 = diff['diff_FGP']
for nan in Z:
    if(nan == 'NaN'):
        nan = 0

columnNames = ['lag5','lag4','lag3','lag2','lag1']
X = pd.DataFrame(np.column_stack((Z[:-5],Z[1:-4],Z[2:-3],Z[3:-2],Z[4:-1])
                                ),columns = columnNames)
iX = pd.DataFrame(np.column_stack((Z[:-5],Z[1:-4],Z[2:-3],Z[3:-2],Z[4:-1])
                                ),columns = columnNames)

print(X.shape)

### We're going to make the win out comes list
### To test for the negative cases we are inversing the data and denoting the losses with a zero for our model
win_count = 0
win_list = []
lose_list = []
while win_count < 76630:
    win_list.append(1)
    lose_list.append(0)
    win_count+=1

#print(win_count)
#print(win_list)


wins = pd.DataFrame(win_list, columns = ['Win'])
loss =  pd.DataFrame(lose_list, columns = ['Win'])



for index, row in iX.iterrows():
    iX.set_value(index,'lag5',-row[0])

#All training data along with inverse Vector differences for losing game scenarios
trainGames = X.append(iX)
results = wins.append(loss)
display(results.shape)

X = trainGames
y = results


#display(trainGames.shape)
    

#display(wins.head(5))
#print(wins.shape)
#display(X.head(5))
#display(iX.head(5))

(76630, 5)


(153260, 1)

In [218]:
from sklearn.model_selection import train_test_split

np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333333)

In [219]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr = linear_model.LinearRegression()

# Fit regression model to the training set
regr.fit(X_train, y_train)

# Apply model to the test set
y_pred = regr.predict(X_test)

# Model evaluation
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print('R-squared = %.4f' % r2_score(y_test, y_pred))

Root mean squared error = 0.4885
R-squared = 0.0455
